In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer



In [2]:
data_path = "Dataset_analyse.csv"
df = pd.read_csv(data_path)

df = df.dropna()
df = df.drop(["title","total_spectator","hebdo_rank","rating_press","rating_public","casting","visa","award","lang"], axis=1)
df

,year,director,country,duration,genre,first_day,first_week,first_weekend,copies,budget
0,1997,3,france,6300,thriller,74295.3622,172230,368944.741401,234,1.549450e+08
1,1999,3,france,5400,drame,75572.2000,154881,412897.600000,198,9.900000e+08
2,1994,3,france,5220,comedie,74295.3622,417021,368944.741401,189,1.549450e+08
3,1995,3,etatsunis,9900,aventure action,56892.4000,355642,286272.200000,302,7.200000e+07
4,1994,3,france,6600,thriller,74295.3622,39739,368944.741401,46,1.549450e+08
...,...,...,...,...,...,...,...,...,...,...
2200,2008,3,grandebretagne,7080,comedie,18126.0000,110442,90372.000000,128,3.072000e+07
2201,1993,3,etatsunis,6060,drame,74295.3622,32351,368944.741401,44,1.549450e+08
2202,1999,3,france,5520,drame,74295.3622,51749,368944.741401,-1,1.549450e+08
2203,1999,3,etatsunis,7080,comedie,74295.3622,63249,368944.741401,-1,1.549450e+08


In [3]:
X = df.drop(['first_week'], axis=1)
y = df.first_week

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore'), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

# param_grid = {'lasso__alpha': [15]}


# model = GridSearchCV(model, param_grid,cv =5)
model.fit(X_train, y_train)
grid_score = model.score(X_train, y_train)

# best_alpha = model.best_params_['lasso__alpha']

/home/utilisateur/Documents/Projets/Brief_Popularity_Film/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9639464724494.12, tolerance: 43318303609.12121
  model = cd_fast.sparse_enet_coordinate_descent(


In [4]:
# y_pred_train = model.predict(X_train)


# residus =  y_pred_train - y_train


# weights = np.exp(-abs(residus) / residus.std())
# model.fit(X_train, y_train, lasso__sample_weight=weights)




y_pred = model.predict(X_test)

In [5]:

r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle :")
# print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle après suppression des valeurs extrêmes:
R2 Score: 0.9402
MAE: 74034.04
RMSE: 117906.43
